In [ ]:
import pypsa
import pandas as pd
import matplotlib.pyplot as plt

Read network

In [ ]:
#n=pypsa.Network("../../results/nz_2050/postnetworks/elec_s_10_ec_lc3.0_Co2L_3H_2050_0.071_AB_0export.nc")
#n=pypsa.Network("../../results/nz_2050_cap/postnetworks/elec_s_10_ec_lc3.0_Co2L0.0_3H_2050_0.071_AB_0export.nc")
#n=pypsa.Network("../../results/nz_nosmr/postnetworks/elec_s_10_ec_lc3.0_Co2L_3H_2030_0.071_AB_0export.nc")
n=pypsa.Network("../../results/nz_2050_fixadj/postnetworks/elec_s_10_ec_lc3.0_Co2L0.0_3H_2050_0.071_AB_0export.nc")
#n=pypsa.Network("../../results/nz_2050_nmain/postnetworks/elec_s_10_ec_lc3.0_Co2L0.0_3H_2050_0.071_AB_0export.nc")
#n=pypsa.Network("../../results/dummy/elec_s_10_ec_lc1.0_Co2L_3H_2030_0.071_AB_120export.nc")

#n=pypsa.Network("../../results/dummy/elec_s_10_ec_lcopt_Co2L-3H.nc")

In [ ]:
n.generators[n.generators.carrier=="onwind"].p_nom_max.sum() /1e3

In [ ]:
n.generators[n.generators.carrier=="solar"].p_nom_max.sum() /1e3

In [ ]:
stats = n.statistics()

In [ ]:
(n.statistics.optimal_capacity()["Generator"]/1e3).round(1) # Capacity in GW

### Dispatch

In [ ]:
(n.statistics.dispatch()[n.statistics.dispatch()>1e3]/ 1e6).round(1)  # Energy in TWh

### Hydro analysis

In [ ]:
n.storage_units

In [ ]:
n.storage_units[n.storage_units.carrier=="hydro"].p_nom.sum()

In [ ]:
n.statistics.capacity_factor(comps=["Generator","StorageUnit"]).round(2)

In [ ]:
n.storage_units_t.inflow.iloc[0:2000].plot() #sum()

In [ ]:
# Total inflow in TWh
n.storage_units_t.inflow.sum().sum() /1e6

In [ ]:
n.statistics.capex()[n.statistics.capex()>1e6] / 1e6

Hydro analysis PyPSA-Earth

In [ ]:
n_earth = pypsa.Network("../../results/pypsa-earth/nz_hydro_eia/networks/elec_s.nc")

In [ ]:
n_earth.storage_units.iloc[0][0:20]

In [ ]:
n_earth.storage_units_t.inflow

### Onshore wind profiles

In [ ]:
n.generators_t.p_max_pu[n.generators[n.generators.carrier=="onwind"].index]

### Statistics

In [ ]:
stats

### Solar PV and onshore wind potentials

Capacity

In [ ]:
n.generators.groupby("carrier").sum().p_nom_max / 1e3 # from MW to GW

### Hydrogen supply and demand

In [ ]:
carrier = "H2"
ebalance = n.statistics.energy_balance().loc[:, :, carrier].groupby(
    "carrier"
).sum().to_frame()

In [ ]:
threshold = 1e3
ebalance = ebalance[ebalance.values<-threshold]
ebalance = ebalance / 1e6 # from MWh to TWh

In [ ]:
ebalance.sum()

In [ ]:
fig, ax = plt.subplots()
ebalance.T.plot.bar(stacked=True, ax=ax, title=carrier, ylabel="TWh")
ax.legend(bbox_to_anchor=(1, 0), loc="lower left", title=None, ncol=1)

### Transport

In [ ]:
ebalance = n.statistics.energy_balance().loc[:, :, "AC"].groupby(
    "carrier"
).sum().to_frame()

In [ ]:
ebalance # BEV charger

In [ ]:
ebalance = ebalance[ebalance.values<-1e3]

### Constraints

In [ ]:
n.global_constraints

### Capacity factors

In [ ]:
n.statistics.capacity_factor(comps=["Generator"])*100

In [ ]:
n.generators_t.p_max_pu[n.generators[n.generators.carrier=="solar"].index].mean()

### Storage capacities

In [ ]:
n.stores .groupby["carrier"].sum() #.e_nom_opt

In [ ]:
n.stores[n.stores.carrier=="H2"].e_nom_opt.sum() / 1e6 # from MWh to TWh

In [ ]:
n.stores[n.stores.carrier=="battery"].e_nom_opt.sum() / 1e6 # from MWh to TWh

In [ ]:
n.stores[n.stores.carrier=="Li ion"].e_nom_opt.sum() / 1e6 # from MWh to TWh

In [ ]:
# 10 TWh h2 and 11 TWh battery

In [ ]:
n.stores.carrier.unique()

In [ ]:
n.storage_units

In [ ]:
n.storage_units.p_nom.sum()